In [10]:
import numpy as np
import statistics

In [2]:
np.mean([1,2,3])

np.float64(2.0)

In [5]:
data= [1,2,3,4]
new = [3]*len(data)

In [14]:
statistics.stdev(data)

1.2909944487358056

In [8]:
list(map(lambda a, b: (a - b)**2, data, new))

[4, 1, 0, 1]

In [1]:
data = [[1,2],[3,4]]

In [2]:
import numpy as np

In [4]:
np.min(data)

np.int64(1)

In [1]:
import pandas as pd
file_name = 'laptop_prices.csv'
col_name = 'Price_euros'

In [2]:
df = pd.read_csv(file_name)

In [3]:
df.shape

(1275, 23)

In [5]:
df['Price_euros']


0       1339.69
1        898.94
2        575.00
3       2537.45
4       1803.60
         ...   
1270     638.00
1271    1499.00
1272     229.00
1273     764.00
1274     369.00
Name: Price_euros, Length: 1275, dtype: float64

# Create latex code for import the data table from xcel file

In [36]:
import pandas as pd
import numpy as np
import os

def get_file_format(filename):
    # Extract the file extension
    if '.' in filename:
        file_extension = filename.rsplit('.', 1)[1].lower()
        return file_extension
    else:
        return None
    
def dataframe_to_latex(df):
    latex_table = "\\begin{tabular}{" + "|c" * len(df.columns) + "|}\n\\hline\n"
    # Header
    header = " & ".join(df.columns) + " \\\\ \\hline\n"
    latex_table += header
    # Rows
    for index, row in df.iterrows():
        line = " & ".join(map(str, row.tolist())) + " \\\\ \\hline\n"
        latex_table += line
    latex_table += "\\end{tabular}"
    return latex_table

In [48]:
folder = './config_sim_data/wald'
filename = 'result_all_wald.xlsx'
pathfile = os.path.join(folder,filename)
sheet_name = 'waldm1sd2n10000'
format = get_file_format(filename)
if format =='xlsx':
    df = pd.read_excel(pathfile,sheet_name = sheet_name)
df = df.round(2)
df.head()

,method,samp/ch_size,error_l,error_r,error_range,n_learn_l,n_learn_r
0,Online BT,50,-0.02,6.36,6.38,118.31,1.57
1,Online BT,100,-0.02,6.36,6.38,231.36,1.40
2,Online BT,500,-0.02,6.36,6.38,609.00,4.00
3,Min-max online BT,50,-0.04,6.36,6.40,165.67,1.33
4,Min-max online BT,100,-0.04,6.96,7.00,248.58,1.00


In [49]:
latex_code = dataframe_to_latex(df)
filename_latex = 'latex_'+sheet_name+'result_all_wald.txt'
pathfile_latex = os.path.join(folder,filename_latex)
# Write latex_code to the specified file
with open(pathfile_latex, 'w') as file:
    file.write(latex_code)

print(f"LaTeX code has been written to {filename}")

LaTeX code has been written to result_all_wald.xlsx


# **Analysing results**

In [1]:
import pickle, os
import pandas as pd
import numpy as np
import openpyxl
from typing import List, Union, Dict
from dataclasses import dataclass, field
import lib_boostrap

In [3]:
@dataclass
class Res_boostrap:
    net_name: str = ''
    net:lib_boostrap.booststream = field(default_factory=lib_boostrap.booststream) 
    chunk_size: int=0
    num_chunk: int =0
    exp_l:List[float] = field(default_factory=list)
    exp_r:List[float] = field(default_factory=list)
    exp_range:List[float] =field(default_factory=list)
    nlearnl:List[float] =field(default_factory=list)
    nlearnr:List[float] =field(default_factory=list)
    
    def add_init_params(self, net:lib_boostrap.booststream, cum:bool=False):
        # add net_name attributes.
        self.net = net
        if self.net.online:
            if not self.net.minmax_boost:
                self.net_name = 'online' if not cum else 'online_cum'
            else:
                self.net_name = 'online_mm' if not cum else 'online_mm_cum'
        else:
            self.net_name = 'offline'
        
            
    
    def add_params(self,net:lib_boostrap.booststream):
        self.net = net
        if self.num_chunk == 0:
            self.chunk_size = self.net.chunk_size
        self.num_chunk +=1
        self.exp_l.append(self.net.exp_l)
        self.exp_r.append(self.net.exp_r)
        self.exp_range.append(self.net.range)
        if len(self.nlearnl) == 0:
            self.nlearnl.append(0)
        else: 
            self.nlearnl.append(self.net.nlearn_l[-1])
        if len(self.nlearnr) == 0:
            self.nlearnr.append(0)
        else:    
            self.nlearnr.append(self.net.nlearn_r[-1])

In [4]:
file_re = './config_sim_data/wiebull/wiebullshape1n10000_re.pkl'

In [5]:
with open(file_re, 'rb') as file:
    loaded_data = pickle.load(file)

In [8]:
res = loaded_data['result_all']

In [22]:
[resname.net_name for resname in res if resname.chunk_size==50]

['online', 'online_mm', 'offline']